In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm, os
import tifffile
import glob

In [24]:
f = glob.glob('D:/allen/data/mydata/ac_time_seperated/20220331/merged/*/*.csv')

In [25]:
f

['D:/allen/data/mydata/ac_time_seperated/20220331/merged\\1\\delrfp.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220331/merged\\1\\gfptrack.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220331/merged\\1\\rfptrack.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220331/merged\\2\\gfptrack.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220331/merged\\3\\rfptrack.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220331/merged\\4\\gfptrack.csv']

In [46]:
# Path to the folder containing the single channel Hoechst microscopy image used for TrackMate. It is important that
# nothing else is present in this folder.
date = '20220331'
scence = '3'

image_folder = r'D:/allen/data/mydata/ac_time_seperated/'+ date +'/merged/' + scence + '/'

# Path to the tracking csv file (rfptrack.cav or gfptrack.csv) generated by TrackMate.
csv_p = image_folder + r"rfptrack.csv"

# Path of the track curation folder which will contain the visualization of the tracks with a potential cell division.
track_curation_folder = image_folder + r'crop_rfp/'

# Track curation: for each track creates images of the detected cells in a folder, to be manually curated for cell division

filenames = os.listdir(image_folder)

saved_folder = r'D:/cycleclassification/2vector_data/rfp/' + date + '/' + scence + '/'

if not os.path.exists(saved_folder):
    os.makedirs(saved_folder)

In [47]:
filenames

['3.tif', 'rfptrack.csv']

In [48]:
img = tifffile.imread(image_folder + filenames[0])[:,:,:,:]

In [49]:

# Define the number of frames to be used
n_frames = img.shape[0]

df = pd.read_csv(csv_p)

# Loop over all tracks and store the track id with potential division in 'split_track.npy'.
if not os.path.exists(track_curation_folder):
    os.mkdir(track_curation_folder)

In [50]:
df

,Label,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,MANUAL_COLOR,MEAN_INTENSITY,MEDIAN_INTENSITY,MIN_INTENSITY,MAX_INTENSITY,TOTAL_INTENSITY,STANDARD_DEVIATION,ESTIMATED_DIAMETER,CONTRAST,SNR
0,ID200,200,0,0.060,166.40,123.76,0,0,0,15,...,-10921639,2.869,0,0,61,30795,5.615,18.645,0.938,0.495
1,ID201,201,1,0.430,142.48,173.68,0,0,0,15,...,-10921639,19.227,0,0,144,206365,25.197,15.153,0.990,0.759
2,ID210,210,0,0.061,167.44,123.76,0,1,1,15,...,-10921639,2.950,0,0,67,31660,5.712,20.131,0.946,0.502
3,ID211,211,1,0.453,139.36,175.76,0,1,1,15,...,-10921639,19.361,0,0,148,207803,25.910,17.054,0.994,0.745
4,ID204,204,1,0.461,136.24,174.72,0,2,2,15,...,-10921639,19.783,0,0,200,212327,26.247,15.567,0.989,0.750
5,ID203,203,0,0.037,165.36,123.76,0,2,2,15,...,-10921639,2.060,0,0,40,22112,4.601,18.349,0.902,0.425
6,ID199,199,1,0.423,128.96,173.68,0,3,3,15,...,-10921639,18.329,0,0,149,196725,24.538,16.491,0.991,0.744
7,ID217,217,1,0.413,124.80,174.72,0,4,4,15,...,-10921639,17.643,0,0,141,189362,23.243,17.253,0.992,0.756
8,ID206,206,1,0.425,120.64,178.88,0,5,5,15,...,-10921639,17.862,2,0,133,191716,22.977,16.833,0.991,0.774
9,ID205,205,1,0.397,117.52,178.88,0,6,6,15,...,-10921639,17.506,4,0,134,187887,21.849,17.029,0.986,0.795


In [51]:
# If the 'TRACK_ID' is not string-type, the ID wilee be ignored in the next part.
for i in range(df.shape[0]):
    df.loc[i,'TRACK_ID'] = str(df.loc[i,'TRACK_ID'])

In [52]:
if not os.path.exists(track_curation_folder + 'split_track.npy'):
    split_track = []
    for track in tqdm.tqdm(np.unique(df['TRACK_ID'])):
        if track != 'None':
            print('track:',track)
            arr = df[df['TRACK_ID'] == str(track)]['FRAME']
            print('arr:',arr)
            frames, nID = np.unique(arr, return_counts=True)
            print(nID)
            division_frame = np.where(nID >= 1)
            #print('division_frame:',division_frame)
            if len(division_frame[0]) > 0:
                if nID[division_frame[0][0]] == nID[-1]:
                    split_track = np.append(split_track, track)
    np.save(track_curation_folder + 'split_track.npy', split_track)
else:
    split_track = np.load(track_curation_folder + 'split_track.npy')

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1002.82it/s]

track: 0
arr: 0    0
2    1
5    2
Name: FRAME, dtype: int64
[1 1 1]
track: 1
arr: 1      0
3      1
4      2
6      3
7      4
8      5
9      6
10     7
11     8
12     9
13    10
14    11
15    12
16    13
17    14
18    15
19    16
20    17
21    18
22    19
23    20
Name: FRAME, dtype: int64
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [53]:
split_track

array(['0', '1'], dtype='<U32')

In [54]:
factor = 3.8496 # 1024/266
crop_size = 70 # half
image_width = 1024 # resolution
for track_id in split_track:
    if not os.path.exists(saved_folder + '{}'.format(track_id)):
        os.makedirs(saved_folder + '{}'.format(track_id))

        print(track_id)

        # Get extreme coordinates of the track for image dimensions
        x_min, x_max = np.int(np.min(df[df['TRACK_ID'] == track_id]['POSITION_X'].values*factor)), \
                       np.int(np.max(df[df['TRACK_ID'] == track_id]['POSITION_X'].values*factor))
        y_min, y_max = np.int(np.min(df[df['TRACK_ID'] == track_id]['POSITION_Y'].values*factor)), \
                       np.int(np.max(df[df['TRACK_ID'] == track_id]['POSITION_Y'].values*factor))

        if np.min([x_min, x_max, y_min, y_max]) > crop_size and np.max([x_min, x_max, y_min, y_max]) < (image_width-crop_size):
            for frame in tqdm.tqdm(np.unique(df[df['TRACK_ID'] == track_id]['FRAME'].values)):
                x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
                y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
                im_bf = img[frame-1,:,y - crop_size:y + crop_size,x - crop_size:x + crop_size]
                print('im_bf:',im_bf.shape)
                tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')

        else:
            for frame in tqdm.tqdm(np.unique(df[df['TRACK_ID'] == track_id]['FRAME'].values)):
                x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
                y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
                if x > crop_size and y < (image_width-crop_size) and x < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size,x - crop_size:x + crop_size]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < 0.5*crop_size or x > (image_width-0.5*crop_size) or y < 0.5*crop_size or y > (image_width-0.5*crop_size):
                    pass  # get rid of cells too close to edge
                elif x > crop_size and x < (image_width-crop_size) and y > (image_width-crop_size):
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width,x - crop_size:x + crop_size]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > crop_size and x < (image_width-crop_size) and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2,x - crop_size:x + crop_size]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size, 0:crop_size*2]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > (image_width-crop_size) and y < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size, image_width -crop_size*2:image_width]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2, 0:crop_size*2]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y > (image_width-crop_size):
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width, 0:crop_size*2]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > (image_width-crop_size) and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2, image_width-crop_size*2:image_width]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                else:
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width, image_width-crop_size*2:image_width]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]<ipython-input-54-d6b00e662556>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
<ipython-input-54-d6b00e662556>:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
100%|█████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 309.65it/s]

0
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
1
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)
im_bf: (6, 140, 140)


In [55]:
import gc

del img, im_bf
gc.collect()

628